In [1]:
%%configure -f
{"executorMemory":"9G","executorCores":5,"numExecutors":6,"driverMemory":"9G",
 "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
     
    }}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1643027560635_0005,pyspark,idle,Link,Link,
7,application_1643027560635_0008,pyspark,shutting_down,Link,Link,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, date_format, unix_timestamp, col

spark = SparkSession \
    .builder \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1643027560635_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def run_queries(df,year):
    # Total revenue
    total_revenue_df = spark.sql("""SELECT SUM(Total_Amt) total_revenue
                               FROM df
                            """)
    total_revenue_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/total_rev')
    # Average trip duration by hour of day
    avg_trip_duration_df = spark.sql("""SELECT H_O_D , AVG(duration) avg_trip_duration
                               FROM df
                               GROUP BY H_O_D
                            """)
    avg_trip_duration_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/avg_dur_hod')
    # Average trip duration by day of week
    avg_trip_duration_df = spark.sql("""SELECT D_O_W , AVG(duration) avg_trip_duration
                               FROM df
                               GROUP BY D_O_W
                            """)
    avg_trip_duration_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/avg_dur_dow')
    # Daily pickup count
    daily_pickup_count_df = spark.sql("""SELECT D_O_Y, COUNT(*) daily_count
                               FROM df
                               GROUP BY D_O_Y
                            """)
    daily_pickup_count_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/daily_pickup_cnt')
    # Average price per pickup
    avg_pickup_price_df = spark.sql("""SELECT SUM(Total_Amt)/COUNT(*) avg_price_pickup
                               FROM df
                               GROUP BY D_O_Y
                            """)
    avg_pickup_price_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/avg_pickup_price_doy')
    # Average trip duration by day of year
    avg_trip_duration_df = spark.sql("""SELECT D_O_Y , AVG(duration) avg_trip_duration
                               FROM df
                               GROUP BY D_O_Y
                            """)
    avg_trip_duration_df.write.mode("overwrite").option("header",True).csv(f's3://nyc-query/{year}/avg_dur_doy')
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for year in range(2009,2022):
    x = 13
    if year == 2021:
        x = 8
    df_list = []
    for month in range(1,x):
        if month < 10:
            month = '0' + str(month)
        df_list.append(spark.read.option("header",True).csv(f's3://nyc-tlc/trip data/yellow_tripdata_{year}-{month}.csv'))
    
    if year == 2016:
        temp_list = []
        for temp_df in df_list:
            temp_df = temp_df.select(['tpep_pickup_datetime', 'tpep_dropoff_datetime','trip_distance','total_amount'])
            temp_list.append(temp_df)
        df_list = temp_list
    df = df_list[0]
    for x in df_list[1:]:
        df = df.union(x)
        
    for c in df.columns:
        col_name = c
        c = c.strip().lower()
        if 'pickup_date' in c:
            df = df.withColumnRenamed(col_name, 'Trip_Pickup_DateTime')
        elif 'dropoff_date' in c:
            df = df.withColumnRenamed(col_name, 'Trip_Dropoff_DateTime')
        elif 'distance' in c:
            df = df.withColumnRenamed(col_name, 'Trip_Distance')
        elif 'total' in c:
            df = df.withColumnRenamed(col_name, 'Total_Amt')
            
    df = df.withColumn('duration', to_timestamp('Trip_Dropoff_DateTime').cast('long')- \
                       to_timestamp('Trip_Pickup_DateTime').cast("long"))
    df = df.withColumn('year',date_format('Trip_Pickup_DateTime','y'))
    df = df.withColumn('Month',date_format('Trip_Pickup_DateTime','M'))
    df = df.withColumn('D_O_Y',date_format('Trip_Pickup_DateTime','D'))
    df = df.withColumn('H_O_D',date_format('Trip_Pickup_DateTime','k'))
    df = df.withColumn('D_O_W',date_format('Trip_Pickup_DateTime','E'))
    
    df = df.select(['Trip_Distance','Total_Amt','year','Month',
                    'D_O_Y','duration','H_O_D','D_O_W'])  
    df.createOrReplaceTempView('df')
    run_queries(df,year)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…